# SQLite3 + SQL Alchemy + Pandas

In [1]:
# Import main libraries
import sqlite3
import pandas as pd
#En caso de no tener instalado sqlalchemy ---> pip install SQLAlchemy
import sqlalchemy as db

## Pasos generales para trabajar con SQLite 3

**1. Crear una conexión a la base de datos:** utilizando el *método connect ()* generamos un objeto de conexión a base de datos SQLite. Si es la primera vez que nos conectamos, se nos creará la base de datos en el mismo directorio donde se este ejecutando el notebook o archivo .py.


**2. Crear un cursor:**  El método *cursor ()* se utiliza para realizar la conexión y ejecutar consultas SQL que nos permiten crear tablas, insertar datos, etc. Para crear un cursor solo necesitamos usar la conexión que ya hemos creado.

**3. Ejecutar una sentencia SQL:** Una vez creado el cursor, podremos ejecutar las sentencias SQL utilizando el método *execute()*.

**4: Realizar un commit:** El método *commit ()* de SQLite se utiliza para guardar cualquier transacción de forma permanente en el sistema de base de datos. Todas las modificaciones de datos o del sistema realizadas por el comando COMMIT desde el comienzo de las transacciones son de naturaleza permanente y no se pueden deshacer ni revertir, ya que una operación COMMIT exitosa libera todos los recursos de transacción involucrados.

**5: Desconectarnos de la base de datos:** El método *close ()* de SQLite cierra la conexión a la base de datos.


## Configuraciones principales

In [2]:
# Crear una conexión a la base de datos
conn = sqlite3.connect('commerce.db')

# Crear un cursor
cursor = conn.cursor()

## Crear una tabla
* Tipos de datos en SQLite --> https://www.sqlite.org/datatype3.html

In [3]:
# Crear una tabla
query = """CREATE TABLE IF NOT EXISTS orders (
            orderId INTEGER,
            totalPrice REAL,
            address TEXT
            )"""

cursor.execute(query)
conn.commit()

## Insertar datos

In [4]:
# Insertar datos
order_data = [
    (546, 3489.98, 'Buenos Aires 989'),
    (547, 6897, 'Espronceda 3218'),
    (560, 7542.35, 'Pasteur 1788'),
    (562, 11432, 'Galicia 1252')
]

cursor.executemany("INSERT INTO orders VALUES (?, ?, ?)", order_data)
conn.commit()

## Seleccionar datos

In [5]:
# Seleccionar un dato utilizando fetchone
query = "SELECT * FROM orders"
cursor.execute(query)
one_result = cursor.fetchone()
print(one_result)

(546, 3489.98, 'Buenos Aires 989')


In [6]:
# Seleccionar varios datos utilizando fetchmany
query = "SELECT * FROM orders"
cursor.execute(query)
many_results = cursor.fetchmany(2)
print(many_results)

[(546, 3489.98, 'Buenos Aires 989'), (547, 6897.0, 'Espronceda 3218')]


In [7]:
# Seleccionar todos los datos utilizando fetchall
query = "SELECT * FROM orders"
cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(546, 3489.98, 'Buenos Aires 989')
(547, 6897.0, 'Espronceda 3218')
(560, 7542.35, 'Pasteur 1788')
(562, 11432.0, 'Galicia 1252')


## Actualizar datos

In [8]:
# Update data
query = """UPDATE orders
            SET totalPrice = 6150.80
            WHERE address = 'Buenos Aires 989'
            
"""
cursor.execute(query)
conn.commit()

# Seleccionar todos los datos utilizando fetchall
query = "SELECT * FROM orders"
cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(546, 6150.8, 'Buenos Aires 989')
(547, 6897.0, 'Espronceda 3218')
(560, 7542.35, 'Pasteur 1788')
(562, 11432.0, 'Galicia 1252')


## Borrar datos

In [9]:
# Borrar datos
query = "DELETE FROM orders WHERE address = 'Pasteur 1788'"
cursor.execute(query)

# Seleccionar todos los datos utilizando fetchall
query = "SELECT * FROM orders"
cursor.execute(query)
results = cursor.fetchall()
for r in results:
    print(r)

(546, 6150.8, 'Buenos Aires 989')
(547, 6897.0, 'Espronceda 3218')
(562, 11432.0, 'Galicia 1252')


## Otros comandos. Descomentar de acuerdo a tu necesidad y ejecutar

In [10]:
# Close connection
# conn.close()

#Drop table
# query = """DROP TABLE IF EXISTS orders"""
# cursor.execute(query)
# conn.commit()

# SQL Alchemy + Pandas

In [11]:
# Si es la primera vez que utilizas SQLAlchemy y no se encuentra instalado, descomentar e instalar.
# pip install sqlalchemy

### Opción 1:Conectarse utilizando un objeto de conexión de SQLite3
Utilizamos el objeto de conexión **conn** definido previamente y luego utilizamos el método read_sql de la librería Pandas para consultar la base de datos y obtener un dataframe.

In [10]:
# Obtener un dataframe a partir de una consulta SQL
query = "SELECT * FROM orders"
data = pd.read_sql(query, con=conn)
data.head()

,orderId,totalPrice,address
0,546,6150.8,Buenos Aires 989
1,547,6897.0,Espronceda 3218
2,562,11432.0,Galicia 1252


### Opción 2: Conectarse utilzando un engine de SQLAlchemy
En este caso vamos a crear un **engine** de SQLAlchemy referenciando la base de datos previamente creada, para luego conectarnos a el.
Luego volvemos a utilizar el método read_sql de la librería Pandas para consultar la base de datos y obtener un dataframe.

In [11]:
# Creación del engine
engine = db.create_engine('sqlite:///commerce.db')
sqla_connection = engine.connect()

# Obtener un dataframe a partir de una consulta SQL
query = "SELECT * FROM orders"
data = pd.read_sql(query, con=sqla_connection)
data.head()

,orderId,totalPrice,address
0,546,6150.80,Buenos Aires 989
1,547,6897.00,Espronceda 3218
2,560,7542.35,Pasteur 1788
3,562,11432.00,Galicia 1252


## Seleccionar datos

In [12]:
# Método: read_sql_table
pd.read_sql_table('orders',con=sqla_connection)

,orderId,totalPrice,address
0,546,6150.80,Buenos Aires 989
1,547,6897.00,Espronceda 3218
2,560,7542.35,Pasteur 1788
3,562,11432.00,Galicia 1252


In [13]:
# Método: read_sql
pd.read_sql_table('orders',con=sqla_connection)

query = "SELECT * FROM orders"
data = pd.read_sql(query, con=sqla_connection)
data

,orderId,totalPrice,address
0,546,6150.80,Buenos Aires 989
1,547,6897.00,Espronceda 3218
2,560,7542.35,Pasteur 1788
3,562,11432.00,Galicia 1252


## Insertar datos

In [18]:
# Para insertar estos datos, previamente descargar el archivo txt que se encuetra en el siguiente link
# https://drive.google.com/file/d/1pJfxW_gUedQlVsO55tVzbQ-FXjk4-jBF/view?usp=sharing'


# Leer el archivo con Pandas. Reemplazar la ruta de la variable path con la de tu archivo.
path = "full_orders.txt"
# Creamos los nombres de las columnas y se la pasamos al csv ya que este no las tiene
header_names = ['orderId', 'totalPrice', 'address']
df = pd.read_csv(path, sep ='\t', names=header_names)
df

,orderId,totalPrice,address
0,1264,12199,Montevideo 3608
1,853,8584,Payro 4273
2,816,9991,Pasteur 4145
3,891,6348,Dominguez 4237
4,1188,3276,Fátima 3916
...,...,...,...
95,1317,12887,La Tribuna 3133
96,1002,13922,Blas Parera 3115
97,441,11933,Pasteur 4884
98,974,14847,Los Pozos 3478


In [16]:
# Previamente vamos a cerrar la conexión conn del objeto SQLite, para utilizar el engine de SQLAlchemy
conn.close()

# Insertar datos en la BD utilzando el método to_sql
df.to_sql(name='orders', con=sqla_connection, if_exists='append', index = False)

99

In [17]:
# Comprobación
query = "SELECT * FROM orders"
data = pd.read_sql(query, con=sqla_connection)
data

,orderId,totalPrice,address
0,546,6150.80,Buenos Aires 989
1,547,6897.00,Espronceda 3218
2,560,7542.35,Pasteur 1788
3,562,11432.00,Galicia 1252
4,853,8584.00,Payro 4273
...,...,...,...
98,1317,12887.00,La Tribuna 3133
99,1002,13922.00,Blas Parera 3115
100,441,11933.00,Pasteur 4884
101,974,14847.00,Los Pozos 3478
